In [ ]:
import numpy as np
import pandas as pd
import keras

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import confusion_matrix, f1_score
import seaborn as sns
from keras import models, layers
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.layers import Input, Concatenate, BatchNormalization
from keras import regularizers
from keras.callbacks import EarlyStopping
from imblearn.over_sampling import SMOTE
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.optimizers import SGD


In [ ]:
# Load dataset
file_path = "concatenated_file.csv"
df = pd.read_csv(file_path)

# Drop non-relevant columns
if "sample_id" in df.columns:
    df.drop(columns=["sample_id"], inplace=True)

In [ ]:
keras.utils.set_random_seed(256)

In [ ]:
# Handle NaN values
numeric_cols = df.select_dtypes(include=[np.number]).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())

# Encode severity as numerical labels
label_encoder = LabelEncoder()
df["severity"] = label_encoder.fit_transform(df["severity"])

In [ ]:
# Extract features and labels
X = df.drop(columns=["severity"]).values
y = to_categorical(df["severity"].values)  # For model training

# Normalize features using MinMaxScaler
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

# Reshape X to fit Conv1D input (samples, time steps, features)
X = X[..., np.newaxis]  # Add dimension for Conv1D

In [ ]:
# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
keras.utils.set_random_seed(256)

LeNet Architecture

In [ ]:
# Define LeNet-style CNN model (1D)
input_shape = (X_train.shape[1], 1)
num_classes = y.shape[1]

model = models.Sequential([
    layers.Input(shape=input_shape),
    # First convolution + average pooling
    layers.Conv1D(filters=6, kernel_size=5, padding='same', activation='sigmoid'),
    layers.AveragePooling1D(pool_size=2, strides=2),
    # Second convolution + average pooling
    layers.Conv1D(filters=16, kernel_size=5, activation='sigmoid'),
    layers.AveragePooling1D(pool_size=2, strides=2),
    # Fully connected layers
    layers.Flatten(),
    layers.Dense(120, activation='sigmoid'),
    layers.Dense(84, activation='sigmoid'),
    layers.Dense(num_classes, activation='softmax')
])

AlexNet Architecture

In [ ]:
model = models.Sequential([
    layers.Input(shape=input_shape),
    layers.Conv1D(filters=16, kernel_size=11, strides=4, activation='relu'),
    layers.MaxPooling1D(pool_size=3, strides=2),
    layers.Conv1D(filters=24, kernel_size=5, padding='same', activation='relu'),
    layers.MaxPooling1D(pool_size=3, strides=2),
    layers.Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'),
    layers.Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'),
    layers.Conv1D(filters=64, kernel_size=3, padding='same', activation='relu'),
    layers.MaxPooling1D(pool_size=3, strides=2),
    layers.Flatten(),
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])


inception Architecture

In [ ]:
# Define Inception block for 1D data with regularization
def inception_1d(x,
                 filters_1x1,
                 filters_3x3_reduce,
                 filters_3x3,
                 filters_5x5_reduce,
                 filters_5x5,
                 filters_pool):
    path1 = layers.Conv1D(filters_1x1, kernel_size=1, padding='same', activation='relu',
                          kernel_regularizer=regularizers.l2(0.01))(x)

    path2 = layers.Conv1D(filters_3x3_reduce, kernel_size=1, padding='same', activation='relu',
                          kernel_regularizer=regularizers.l2(0.01))(x)

    path2 = layers.Conv1D(filters_3x3, kernel_size=3, padding='same', activation='relu',
                          kernel_regularizer=regularizers.l2(0.01))(path2)


    path3 = layers.Conv1D(filters_5x5_reduce, kernel_size=1, padding='same', activation='relu',
                          kernel_regularizer=regularizers.l2(0.01))(x)

    path3 = layers.Conv1D(filters_5x5, kernel_size=5, padding='same', activation='relu',
                          kernel_regularizer=regularizers.l2(0.01))(path3)


    path4 = layers.MaxPooling1D(pool_size=3, strides=1, padding='same')(x)
    path4 = layers.Conv1D(filters_pool, kernel_size=1, padding='same', activation='relu',
                          kernel_regularizer=regularizers.l2(0.01))(path4)


    return layers.Concatenate()([path1, path2, path3, path4])

#%%
# Define the model
input_shape = (X_train.shape[1], 1)
num_classes = y.shape[1]
input_layer = layers.Input(shape=input_shape)

# First Inception block
x = inception_1d(input_layer,
                 filters_1x1=16,
                 filters_3x3_reduce=8,
                 filters_3x3=16,
                 filters_5x5_reduce=8,
                 filters_5x5=16,
                 filters_pool=8)
x = layers.Dropout(0.4)(x)  # Increased dropout

# Auxiliary classifier
aux1 = layers.AveragePooling1D(pool_size=5, strides=3, padding='valid')(x)
aux1 = layers.Conv1D(832, kernel_size=1, padding='same', activation='relu')(aux1)
aux1 = layers.Flatten()(aux1)
aux1 = layers.Dense(832, activation='relu')(aux1)
aux1 = layers.Dropout(0.4)(aux1)
aux1 = layers.Dense(num_classes, activation='softmax', name='aux_output')(aux1)

# Second Inception block
x = inception_1d(x,
                 filters_1x1=16,
                 filters_3x3_reduce=8,
                 filters_3x3=16,
                 filters_5x5_reduce=16,
                 filters_5x5=16,
                 filters_pool=8)
x = layers.Dropout(0.4)(x)  # Increased dropout

# Main output
x = layers.Flatten()(x)
x = layers.Dropout(0.4)(x)  # Increased dropout
output_layer = layers.Dense(num_classes, activation='softmax', name='main_output')(x)

# Create model with two outputs
model = models.Model(inputs=input_layer, outputs=[output_layer ,aux1])

For LeNet and AlexNet

In [ ]:
# Compile model
optimizer = Adam(learning_rate=0.001, clipnorm=1.0)
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["categorical_accuracy"])

For Inception

In [ ]:
# Compile model
optimizer = Adam(learning_rate=0.001, clipnorm=1.0)
model.compile(loss=["categorical_crossentropy", "categorical_crossentropy"],
             loss_weights=[0.8, 0.2],
             optimizer=optimizer,
             metrics=["categorical_accuracy","categorical_accuracy"])

Train For leNet and AlexNet

In [ ]:
# Train model
early_stopping = EarlyStopping(monitor='val_loss',patience=3,restore_best_weights=True)

# Add callback to your fit method
history = model.fit(X_train, y_train, batch_size=30, epochs=10, validation_split=0.1, callbacks=[early_stopping] 

Train for Inception

In [ ]:
# Train model with Early Stopping
from keras import backend as K
K.clear_session()
early_stopping = EarlyStopping(monitor='val_main_output_loss', patience=5, restore_best_weights=True, mode='min')
history = model.fit(X_train, [y_train, y_train], batch_size=30, epochs=20, validation_split=0.2, callbacks=[early_stopping])

(Evaluation + graphs ) for LeNet and AlexNe

In [ ]:
# Evaluate model
score = model.evaluate(X_test, y_test, verbose=0)
test_loss = score[0]   # Adjust loss as per original code
test_accuracy = score[1]  # Adjust accuracy as per original code
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

# Plot Loss & Accuracy curves
plt.figure(figsize=(12, 5))

# Plot Loss Curve
plt.subplot(1, 2, 1)
plt.plot(history.history.get('loss', []), label='Loss Entraînement', color='blue')
plt.plot(history.history.get('val_loss', []), label='Loss Validation', color='red')
plt.axhline(y=test_loss, color='green', linestyle='--', label='Loss Test')
plt.xlabel('Époques')
plt.ylabel('Perte (Loss)')
plt.title('Évolution de la Loss')
plt.legend()

# Plot Accuracy Curve
plt.subplot(1, 2, 2)
plt.plot(history.history.get('categorical_accuracy', []), label='Accuracy Entraînement', color='blue')
plt.plot(history.history.get('val_categorical_accuracy', []), label='Accuracy Validation', color='red')
plt.axhline(y=test_accuracy, color='green', linestyle='--', label='Accuracy Test')
plt.xlabel('Époques')
plt.ylabel('Précision (Accuracy)')
plt.title("Évolution de l'Accuracy")
plt.legend()

# Save and display plots
#plt.savefig('training_curves_no_normalization.png')
plt.show()

For Inception

In [ ]:
# Evaluate model and plot Loss & Accuracy curves
# Évaluer le modèle sur les données de test
score = model.evaluate(X_test, [y_test, y_test], verbose=0)

# Afficher les résultats de l'évaluation (using main output metrics)
print("Test loss:", score[2])  # Affiche la perte sur l'ensemble de test (combined loss)
print("Test accuracy:", score[4])  # Affiche la précision sur l'ensemble de test (main_output accuracy)

print(score)
# Calculate adjusted test loss and accuracy for plotting
test_loss = score[2]   # Adjust loss as per original code
test_accuracy = score[4]   # Adjust accuracy as per original code

# Plot Loss & Accuracy curves
plt.figure(figsize=(12, 5))

# Plot Loss Curve (using main_output loss)
plt.subplot(1, 2, 1)
plt.plot(history.history.get('main_output_loss', []), label='Loss Entraînement', color='blue')
plt.plot(history.history.get('val_main_output_loss', []), label='Loss Validation', color='red')
plt.axhline(y=test_loss, color='green', linestyle='--', label='Loss Test')
plt.xlabel('Époques')
plt.ylabel('Perte (Loss)')
plt.title('Évolution de la Loss')
plt.legend()

# Plot Accuracy Curve (using main_output categorical_accuracy)
plt.subplot(1, 2, 2)
plt.plot(history.history.get('main_output_categorical_accuracy', []), label='Accuracy Entraînement', color='blue')
plt.plot(history.history.get('val_main_output_categorical_accuracy', []), label='Accuracy Validation', color='red')
plt.axhline(y=test_accuracy, color='green', linestyle='--', label='Accuracy Test')
plt.xlabel('Époques')
plt.ylabel('Précision (Accuracy)')
plt.title("Évolution de l'Accuracy")
plt.legend()

# Save and display plots
#plt.savefig('training_curves_inception_adjusted.png')
plt.show()